# Operational workflow

This is an example of an operational workflow using DELFT3D and the NOAA's operational gfs data provided for the last 10 days at http://nomads.ncep.noaa.gov

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.model as pm
from pyPoseidon.utils import data,point

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import datetime

hv.notebook_extension('bokeh')

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':160., # lat/lon window
     'maxlon':179.,
     'minlat':-49.,
     'maxlat':-33.,
     'solver':'d3d',
     'meteo': 'gfs_oper', 
     'resolution':0.2, #grid resoltuion 
     'step':20, # step for output of map field in d3d
     'start_date': datetime.datetime.strftime(datetime.date.today()-datetime.timedelta(1), '%Y-%m-%d'), #yesterday -> 2 days ahead
     'time_frame':'2d',
 #    'end_date':'2013-10-30' optional end date instead of time_frame
     'ncores': 4 , #number of cores
     'rpath':'./test/', #location of calc folder
     'epath':'/Users/brey/DELFT3D/SVN/7545/bin/lnx64/', #folder for solver executables
     'conda_env':'mpi3' # optional conda env for running the solver,
    }

### Create model

In [ ]:
#initialize a model
b = pm.model(**dic)

### set it up

In [ ]:
b.set() #set it up 

### output to rpath 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

In [ ]:
# save all matrices for further use
#b.pickle(path=path)

### execute

In [ ]:
#execute
b.run()

### Visualise output

In [ ]:
otp = data(**dic)

#### Grid & Bathymetry 

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

otp.impl.dem.bathymetry.plot.pcolormesh('longitude', 'latitude', ax=ax);

gp = ax.scatter(otp.impl.grid.lons, otp.impl.grid.lats, s=2, color='k',  label='grid points', transform=ccrs.PlateCarree());

pp = ax.scatter(otp.impl.xh, otp.impl.yh, s=2, color='b', label='pressure points', transform=ccrs.PlateCarree());

ax.legend()

ax.coastlines('50m'); ax.gridlines(draw_labels=True);

#### Variable snapshot

In [ ]:
otp.impl.Dataset.S1[10,:,:].plot() 

#### Holoviews

In [ ]:
%%opts Image [width=650 height=450 colorbar=True] (cmap='jet')
#Fast but no map
otp.impl.hview('S1').to(hv.Image, ['XZ','YZ'])#.hist()

#### Geoviews

In [ ]:
#with map and zooming but dynamic colorbar to make it fast, otherwise slow and heavy
g50 = gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
%%opts Image [colorbar=True width=650 height=450 toolbar="above"  tools=['hover']] (cmap='viridis')
otp.impl.gview('S1').to(gv.Image, ['XZ','YZ']) * gf.coastline() * gf.borders()

#### Inherent Animation option

In [ ]:
# relatively fast and you can save in mp4
otp.impl.frames(['S1'],title='SSH')

In [ ]:
# relatively fast and you can save in mp4
otp.impl.frames(['U1','V1'],title='Sea Vectors',scale=.1)#, savepath='')

### Observation points

In [ ]:
otp.impl.obs.locations # in-situ measurements available within the lat/lon window

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(2)
p.head()

In [ ]:
# .... or get with Station Name  
p = otp.impl.obs.loc('New Zealand - Castlepoint')
p.head()

In [ ]:
#plot
ax = p.plot()

#### sample nearest point from simulation 

In [ ]:
lat,lon = otp.impl.obs.locations.loc[2,['lat','lon']] #get lat lon from tide gauge
print lat, lon

In [ ]:
#optional, not always acurate due to nan
#ts = otp.vars.S1.sel(XZ=[lon], method='nearest').sel(YZ=[lat], method='nearest')
#otp.vars.XZ.sel(XZ=[lon], method='nearest'), otp.vars.YZ.sel(YZ=[lat], method='nearest')

#Check ts, could be nan
#ts = ts.squeeze().to_pandas()
#ts.head()

In [ ]:
ts = pyPoseidon.utils.point(lon=lon,lat=lat,data=otp.impl)
ts.tseries(var='S1')

In [ ]:
ts.S1.plot()

In [ ]:
## Join the graphs
ax = ts.S1.plot(figsize=(10,8),color=['r'])
p.plot(ax=ax)
ax.legend(['D3D'] + list(p.columns.values) , loc='center left',bbox_to_anchor=(1.1, 0.5))